In [7]:
import os 
from dotenv import load_dotenv

os.environ['HF_HOME']='/datasets/cc-20250630151645'
os.environ['HF_HUB_CACHE']='/datasets/cc-20250630151645'
os.environ['TRANSFORMERS_CACHE']='/datasets/cc-20250630151645'
os.environ['NEMO_HOME']='/datasets/cc-20250630151645'

load_dotenv()
print(os.getenv('NVIDIA_API_TOKEN'))

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


In [4]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Send one page of a PDF to an NVIDIA/OpenAI multimodal model
using pdf2image + PyMuPDF backend (no poppler needed).
"""

import fitz  # pip install pymupdf
from pathlib import Path
import base64, os, io
from openai import OpenAI
from PIL import Image
from pdf2image import convert_from_path     # uses PyMuPDF when use_fitz=True

# ------------------------------------------------------------------ #
# 1. Extract *one page* with PyMuPDF backend
# ------------------------------------------------------------------ #



def pdf_page_to_base64(pdf_path: str | Path, page_index: int = 0) -> str:
    """
    Render a single PDF page with PyMuPDF and return base-64 PNG bytes.
    """
    pdf_path = Path(pdf_path)
    doc = fitz.open(pdf_path)
    if page_index < 0 or page_index >= doc.page_count:
        raise ValueError(f"Invalid page index {page_index} for {pdf_path}")

    page = doc.load_page(page_index)
    pix  = page.get_pixmap(dpi=200)      # adjust DPI if needed

    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    return base64.b64encode(buf.getvalue()).decode("utf-8")

# ------------------------------------------------------------------ #
# 2. Build the correct multimodal message
# ------------------------------------------------------------------ #
def make_mm_message(text: str, b64_png: str) -> list[dict]:
    """
    Returns the message array expected by OpenAI/NVIDIA multimodal chat.
    """
    return [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": text},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{b64_png}",
                        "detail": "auto",
                    },
                },
            ],
        }
    ]

# ------------------------------------------------------------------ #
# 3. Send to NVIDIA / OpenAI multimodal endpoint
# ------------------------------------------------------------------ #
def send_pdf_page(
    pdf_file    : str | Path,
    page_idx    : int,
    user_prompt : str,
    model       : str = "nvidia/llama-3.1-nemotron-nano-vl-8b-v1",
    api_key     : str | None = os.getenv("NVIDIA_API_TOKEN"),
    base_url    : str = "https://integrate.api.nvidia.com/v1",
    temperature : float = 0.7,
    top_p       : float = 0.95,
    max_tokens  : int = 1024,
):
    if not api_key:
        raise RuntimeError("Set NVIDIA_API_TOKEN (or pass api_key=…)")

    client = OpenAI(base_url=base_url, api_key=api_key)

    img_b64   = pdf_page_to_base64(pdf_file, page_idx)
    messages  = make_mm_message(user_prompt, img_b64)

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        stream=True,
    )

    print("---- Response ----")
    for chunk in completion:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
    print()

# ------------------------------------------------------------------ #
# Example
# ------------------------------------------------------------------ #
if __name__ == "__main__":
    PDF_PATH   = (
        "/home/jovyan/datasets/cc-20250630151645/src/Guidelines/"
        "Clinical Pharmacogenetics Implementation Consortium (CPIC) "
        "Guideline for CYP2B6 and Efavirenz-containing Antiretroviral "
        "Therapy (April 2019) .pdf"
    )
    PAGE_INDEX = 1
    PROMPT     = "Extract table from the page using markdown format."

    send_pdf_page(PDF_PATH, PAGE_INDEX, PROMPT)


---- Response ----
Table 1 Assignment of likely CYP2B6 phenotypes based on genotypes

created to allow for the possibility that these may be clinically relevant for efavirenz or other CYP2B6 substrates, such as bupropion and methadone. See the CYP2B6 Diplotype-Phenotype Table1 for a complete list of possible diplotypes and phenotype assignments.

Genetic test interpretation

Many clinical laboratories report CYP2B6 genotype results using the star-allele (*) nomenclature. The (*)-allele nomenclature for CYP2B6 alleles is found at the Pharmacogene Variation (PharmVar) Consortium website (https://www.pharmvar.org/ gene/CYP2B6). Some laboratories test and report only on specific SNPs that have been most extensively studied, such as c.516G>T and c.983T>C (rs28399499, p.1328T). These variants are the only defining SNPs for CYP2B6 * 9 and * 18, respectively, but c.983T>C is also found in combination with another variant in * 16, and c.516G>T is found in combination with other variants in 11 o